In [1]:
!curl http://localhost:9200/

{
  "name" : "node-1",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "Hd5owN6ERfSiT3DlTZTENg",
  "version" : {
    "number" : "8.2.2",
    "build_flavor" : "default",
    "build_type" : "deb",
    "build_hash" : "9876968ef3c745186b94fdabd4483e01499224ef",
    "build_date" : "2022-05-25T15:47:06.259735307Z",
    "build_snapshot" : false,
    "lucene_version" : "9.1.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:
import elasticsearch 
import json

from elasticsearch import helpers
from collections import deque

In [3]:
def data_gen(): 
    with open('../data_hub/data.json', 'r') as f:
        documents = json.load(f)  
           
    for doc in documents:
        json_data = {}
        json_data['article_title'] = doc['article_title']
        json_data['section_title'] = doc['section_title']
        json_data['main_section'] = doc['main_section']
        json_data['section_number'] = doc['section_number']
        json_data['page_id'] = doc['page_id']
        json_data['text'] = doc['text']
        json_data['source_url'] = doc['source_url']
        json_data['tags'] = doc['tags']
        yield json_data 

In [4]:
# Initiate Elasticsearch
es = elasticsearch.Elasticsearch('http://127.0.0.1:9200/')

# Bulk load data into Elasticsearch
es.indices.create(index='pandemic_docs')
deque(helpers.parallel_bulk(es, data_gen(), index='pandemic_docs'), maxlen=0)
es.indices.refresh()

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [5]:
# Search documents in Elasticsearch
query = {'query':{'match_phrase':{'text':'Plague of Cyprian'}}}

hits = es.search(index='pandemic_docs', body=query) 
hits['hits']

/tmp/ipykernel_28308/1453526317.py:4: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  hits = es.search(index='pandemic_docs', body=query)


{'total': {'value': 9, 'relation': 'eq'},
 'max_score': 11.911831,
 'hits': [{'_index': 'pandemic_docs',
   '_id': 'y4kCVIEB5DPWiFiNugmg',
   '_score': 11.911831,
   '_ignored': ['text.keyword'],
   '_source': {'article_title': 'Plague of Cyprian',
    'section_title': 'Epidemiology',
    'main_section': 'Epidemiology',
    'section_number': 2,
    'page_id': 16856479,
    'text': "Plague of Cyprian,Epidemiology\nThe severe devastation to the European population from the two plagues may indicate that the people had no previous exposure or immunity to the plague's cause. The historian William Hardy McNeill asserts that both the earlier Antonine Plague (166–180) and the Plague of Cyprian (251–270) were the first transfers from animal hosts to humanity of two different diseases, one of smallpox and one of measles, but not necessarily in that order. D. Ch. Stathakopoulos asserts that both outbreaks were of smallpox.According to the historian Kyle Harper, the symptoms attributed by ancient 

In [6]:
es.cat.count(index='_all', format='json')

ListApiResponse([{'epoch': '1654971883', 'timestamp': '18:24:43', 'count': '401'}])

In [7]:
es.cat.count(index='pandemic_docs', format='json')

ListApiResponse([{'epoch': '1654971885', 'timestamp': '18:24:45', 'count': '401'}])